In [69]:
include("Acrobots/src/Acrobots.jl")

Acrobots

In [70]:
robot = Acrobots.acrobot()
state = Acrobots.AcrobotState{Float64}(0.)
input = Acrobots.AcrobotInput{Float64}(0.)

Acrobots.AcrobotInput{Float64}(0.0)

In [71]:
vis = Acrobots.DrakeVisualizer(robot)
state = Acrobots.AcrobotState{Float64}([pi/4;pi/4;0;0])
Acrobots.draw(vis, state)

In [72]:
linear_sys = Acrobots.linearize(robot, 0, state, input)

Acrobots.LinearSystem{Float64,Acrobots.AcrobotState{T},Acrobots.AcrobotInput{T},Acrobots.AcrobotOutput{T},4,1,4}(FixedSizeArrays.Mat{4,4,Float64}(
    0.0 0.0 1.0 0.0
    0.0 0.0 0.0 1.0
    -10.871914666045658 -1.1897211413655526 -0.10448664063666142 0.160037927958154
    16.6520684892354 -1.0605862390369154 0.160037927958154 -0.32031154006812
)
,FixedSizeArrays.Mat{4,1,Float64}(
    0.0
    0.0
    -1.60037927958154
    3.2031154006812
)
,FixedSizeArrays.Mat{4,4,Float64}(
    1.0 0.0 0.0 0.0
    0.0 1.0 0.0 0.0
    0.0 0.0 1.0 0.0
    0.0 0.0 0.0 1.0
)
,FixedSizeArrays.Mat{4,1,Float64}(
    0.0
    0.0
    0.0
    0.0
)
)

In [73]:
Acrobots.dynamics(linear_sys, 0, state, input)

Acrobots.AcrobotState{Float64}(0.0,0.0,-9.473186610709663,12.245521523949805)

In [74]:
Acrobots.dynamics(robot, 0, Acrobots.AcrobotState{Float64}([1.; 2.;3.; 4.]), Acrobots.AcrobotInput{Float64}(5.))

Acrobots.AcrobotState{Float64}(3.0,4.0,21.382653016334,-18.427570843977477)

In [75]:
Acrobots.output(robot, 0, Acrobots.AcrobotState{Float64}([1.; 2.; 3.; 4.]), Acrobots.AcrobotInput{Float64}(5.))

Acrobots.AcrobotOutput{Float64}(1.0,2.0,3.0,4.0)

In [81]:
using ProfileView


xbar = Acrobots.AcrobotState{Float64}([pi; 0; 0; 0])
state = Acrobots.AcrobotState{Float64}([[pi; 0] + 0.1*(rand(2)-0.5); zeros(2)])
robot = Acrobots.acrobot()
input = Acrobots.AcrobotInput{Float64}(0)
dt = 0.001
vis = Acrobots.DrakeVisualizer(robot)
ts = 0:dt:5
# for (i, t) in enumerate(ts[1:100])
#     statedot = Acrobots.dynamics(robot, t, state, input)
#     state += statedot * dt
#     Acrobots.draw(vis, state)
# end
# Profile.clear()

lqr_state = Acrobots.LQRState{Float64}(0)

controller = Acrobots.lqr(Acrobots.linearize(robot, 0, xbar, input), diagm([10,10,1,1]), eye(1))

elapsed = @elapsed for (i, t) in enumerate(ts)
    output = Acrobots.output(robot, t, state, input)
    
    controller_input = output - Acrobots.AcrobotOutput(xbar)
    input = Acrobots.output(controller, t, lqr_state, controller_input)
#     @show input
    statedot = Acrobots.dynamics(robot, t, state, input)
#     linear_system = Acrobots.linearize(robot, t, state, input)
    
#     perturbation = 1e-4 * Acrobots.AcrobotState{Float64}(1)
#     perturbed_state = state + perturbation
    
    
#     @assert isapprox(convert(Vector, Acrobots.dynamics(robot, t, perturbed_state, input)), 
#     convert(Vector, statedot + Acrobots.dynamics(linear_system, t, perturbation, Acrobots.AcrobotInput{Float64}(0))), atol=1e-6)
    state += statedot * dt
    Acrobots.draw(vis, state)
end
@show elapsed / length(ts)
# ProfileView.view()

sys.A = FixedSizeArrays.Mat{4,4,Float64}(
    0.0 0.0 1.0 0.0
    0.0 0.0 0.0 1.0
    12.629158094942365 -12.692621200947098 -0.17208140873863043 0.3014659265872247
    -14.748825835500533 29.61188526180959 0.30146592658722476 -0.6033200067279952
)

sys.B = FixedSizeArrays.Mat{4,1,Float64}(
    0.0
    0.0
    -3.014659265872247
    6.033200067279952
)

K = [-278.44223125457574 -112.29125984465645 -119.72457376720195 -56.828240171727884]
elapsed / length(ts) = 0.0007198309136172765

0.0007198309136172765